In [ ]:
# !pip install -r requirements.txt

In [ ]:
!pip install cryptography>=36.0.0
!pip install pdfservices-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 14.6 MB/s eta 0:00:00
  Created wheel for polling: filename=polling-0.3.2-py3-none-any.whl s

# New Section

In [ ]:
from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType
from adobe.pdfservices.operation.client_config import ClientConfig
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_renditions_element_type import ExtractRenditionsElementType
from adobe.pdfservices.operation.pdfops.options.extractpdf.table_structure_type import TableStructureType
import os.path
import zipfile
import json

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
# #Initial setup, create credentials instance.
# credentials = Credentials.service_account_credentials_builder()\
#     .from_file(r'/content/pdfservices-api-credentials.json') \
#     .build()

# client_config = ClientConfig.builder().with_connect_timeout(10000).with_read_timeout(40000).build()

# #Create an ExecutionContext using credentials and create a new operation instance.
# execution_context = ExecutionContext.create(credentials,client_config)


In [ ]:
import os
os.environ['PDF_SERVICES_CLIENT_SECRET'] = "p8e-DU5_VLKpptOpBqIwi6d-YkacgAQV5jUi"
os.environ['PDF_SERVICES_CLIENT_ID'] = "f60bced5ede94a8ca736da2025a4dede"

In [ ]:
import os
from tqdm import tqdm
import logging

In [ ]:
def create_json_files(input_path = '/content/'):
  credentials = Credentials.service_principal_credentials_builder(). \
        with_client_id(os.getenv('PDF_SERVICES_CLIENT_ID')). \
        with_client_secret(os.getenv('PDF_SERVICES_CLIENT_SECRET')). \
        build()

  execution_context = ExecutionContext.create(credentials)
  extract_pdf_operation = ExtractPDFOperation.create_new()
  input_files = os.listdir()

  input_pdfs = [pdf for pdf in input_files if(pdf.split('.')[-1] == 'pdf')]

  for pdf in tqdm(input_pdfs):

    zip_file = "./Extracted_"+str(pdf.split('.')[0])+'.zip'

    if os.path.isfile(zip_file):
        os.remove(zip_file)

    input_path = r'/content/'+pdf

    extract_pdf_operation = ExtractPDFOperation.create_new()

    #Set operation input from a source file.
    source = FileRef.create_from_local_file(input_path)
    extract_pdf_operation.set_input(source)

    # Build ExtractPDF options and set them into the operation
    extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
              .with_elements_to_extract([ExtractElementType.TEXT, ExtractElementType.TABLES]) \
              .with_element_to_extract_renditions(ExtractRenditionsElementType.TABLES) \
              .with_table_structure_format(TableStructureType.CSV) \
              .build()
    # extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
    #       .with_element_to_extract(ExtractElementType.TEXT) \
    #       .with_include_styling_info(True) \
    #       .build()
    extract_pdf_operation.set_options(extract_pdf_options)

    #Execute the operation.
    result: FileRef = extract_pdf_operation.execute(execution_context)

    #Save the result to the specified location.
    result.save_as(zip_file)

In [ ]:
create_json_files()

100%|██████████| 1/1 [01:07<00:00, 67.01s/it]


In [ ]:
# Unzip result file
import json
import zipfile
import os
with zipfile.ZipFile("/content/Extracted_NCT01396148.zip", 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())

with open('structuredData.json') as json_file:
    data = json.load(json_file)


In [ ]:
import pandas as pd
elements_data = pd.json_normalize(data['elements'])

In [ ]:
toc_data = elements_data[elements_data['Path'].str.contains('TOC')]

,Bounds,Page,Path,attributes.BBox,attributes.Placement,HasClip,Lang,Text,TextSize,Font.alt_family_name,...,attributes.InlineAlign,attributes.RowIndex,attributes.Width,attributes.BorderColor,attributes.BorderStyle,attributes.BorderThickness,ClipBounds,attributes.RowSpan,attributes.ColSpan,attributes.BackgroundColor
444,"[89.94999694824219, 669.9100036621094, 543.0, ...",10.0,//Document/TOC/TOCI/Span/Reference,NaN,NaN,False,NaN,LIST OF TABLES...................................,12.0,Times New Roman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445,"[90.0, 655.4759979248047, 98.9952392578125, 67...",10.0,//Document/TOC/TOCI[2]/Reference/Lbl,NaN,NaN,False,NaN,1.,12.0,Times New Roman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
446,"[98.9952392578125, 655.4759979248047, 543.0, 6...",10.0,//Document/TOC/TOCI[2]/Reference/LBody,NaN,NaN,False,NaN,INTRODUCTION ...................................,12.0,Times New Roman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447,"[118.80000305175781, 635.6759948730469, 136.89...",10.0,//Document/TOC/TOCI[3]/Reference/Lbl,NaN,NaN,False,NaN,1.1.,12.0,Times New Roman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448,"[136.8959503173828, 635.6759948730469, 543.0, ...",10.0,//Document/TOC/TOCI[3]/Reference/LBody,NaN,NaN,False,NaN,Indication......................................,12.0,Times New Roman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,"[89.94999694824219, 624.3099975585938, 543.0, ...",14.0,//Document/TOC[3]/TOCI[3]/Span/Reference,NaN,NaN,False,NaN,Appendix 3. Blood Pressure Levels for Girls an...,12.0,Times New Roman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689,"[89.94999694824219, 604.5099945068359, 543.0, ...",14.0,//Document/TOC[3]/TOCI[4]/Span/Reference,NaN,NaN,False,NaN,Appendix 4. Response Evaluation Criteria in So...,12.0,Times New Roman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
690,"[89.94999694824219, 584.7100067138672, 543.0, ...",14.0,//Document/TOC[3]/TOCI[5]/Span/Reference,NaN,NaN,False,NaN,Appendix 5. Lansky and ECOG Performance Scales...,12.0,Times New Roman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
691,"[89.94999694824219, 564.9100036621094, 543.0, ...",14.0,//Document/TOC[3]/TOCI[6]/Span/Reference,NaN,NaN,False,NaN,Appendix 6. Sunitinib Total Daily Dose Levels ...,12.0,Times New Roman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
